# Uniview module for [M82](https://en.wikipedia.org/wiki/Messier_82) 

I will use the  [FIRE](https://galaxies.northwestern.edu/fire-simulations/) isolated galaxy model that we've used for testing [Firefly](https://github.com/ageller/Firefly) here.  That simulation’s “proper name” is control_G4_020 and was run by Paul Torrey as part of his nuclear star formation paper.  Note, the bi-conical outflow in the simulation  is due to AGN feedback, so scientifically this isn’t accurate.  But the look should be fine for our purposes. 

*Aaron Geller, 2019*

### Imports and function definitions

In [4]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data"

import sys, os, shutil, errno, string, urllib

sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir) )))
#print(sys.path)
import uvmodlib.v1 as uvmod

### USES Conf Templates

In [77]:
GalaxyTemplate = """mesh
{  
    data gas    ./modules/$folderName/FIREgas.raw
    data stars    ./modules/$folderName/FIREstars.raw

    cullRadius $cr
    glslVersion 330
    enableTypeCheck true
    
    propertyCollection 
    {        
        __objectName__
        {           

            
            vec1f gasAlpha 0.1 | public | desc "gas alpha" | widget slider | range 0 1
            vec1f gasPsize 0.2 | public | desc "gas particle size" | widget slider | range 0 10
            vec1f gasHighVPsize 2 | public | desc "gas particle size  for v > vMax" | widget slider | range 0 10
            vec1f gasHighVAlpha 0.07 | public | desc "gas particle alpha for v > vMax" | widget slider | range 0 10
            vec1f gasVMin 0. | public | desc "gas velocity colormap min" | widget slider | range 0 500
            vec1f gasVMax 500. | public | desc "gas velocity colormap max" | widget slider | range 500 1000
            
            vec1f starsAlpha 0.015 | public | desc "stars alpha" | widget slider | range 0 1
            vec1f starsPsize 0.80 | public | desc "stars particle size" | widget slider | range 0 10
            vec1f starsHighVPsize 0.80 | public | desc "stars particle size for v > vMax" | widget slider | range 0 10
            vec1f starsHighVAlpha 0.015 | public | desc "stars particle alpha for v > vMax" | widget slider | range 0 10
            vec1f starsVMin 0. | public | desc "stars velocity colormap min" | widget slider | range 0 500
            vec1f starsVMax 400. | public | desc "stars velocity colormap max" | widget slider | range 500 1000
            
            vec1f rotationX 2.5 | public | desc "X rotation angle" | widget slider | range 0 6.283
            vec1f rotationY 2.0 | public | desc "Y rotation angle" | widget slider | range 0 6.283
            vec1f rotationZ 0.0 | public | desc "Z rotation angle" | widget slider | range 0 6.283
        }
    }


   
    ############# gas particles
    pass
    {
        useDataObject gas
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/galaxy.gs
                vertexShader ./modules/$folderName/galaxy.vs
                fragmentShader   ./modules/$folderName/galaxy.fs
                texture cmap ./modules/$folderName/cmapReds.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    #colorspace linear
                } 
                stateManagerVar __objectName__.gasPsize  userPsize
                stateManagerVar __objectName__.gasAlpha  userAlpha
                stateManagerVar __objectName__.rotationX  userRotationX
                stateManagerVar __objectName__.rotationY  userRotationY
                stateManagerVar __objectName__.rotationZ  userRotationZ
                
                stateManagerVar __objectName__.gasVMin  vMin
                stateManagerVar __objectName__.gasVMax  vMax
                stateManagerVar __objectName__.gasHighVPsize  highVPsize
                stateManagerVar __objectName__.gasHighVAlpha  highVAlpha
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }  
    }
    
    ############# stars particles
    pass
    {
        useDataObject stars
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/galaxy.gs
                vertexShader ./modules/$folderName/galaxy.vs
                fragmentShader   ./modules/$folderName/galaxy.fs
                texture cmap ./modules/$folderName/cmapBlues.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    #colorspace linear
                } 
                stateManagerVar __objectName__.starsPsize  userPsize
                stateManagerVar __objectName__.starsAlpha  userAlpha
                stateManagerVar __objectName__.rotationX  userRotationX
                stateManagerVar __objectName__.rotationY  userRotationY
                stateManagerVar __objectName__.rotationZ  userRotationZ

                stateManagerVar __objectName__.starsVMin  vMin
                stateManagerVar __objectName__.starsVMax  vMax
                stateManagerVar __objectName__.starsHighVPsize  highVPsize
                stateManagerVar __objectName__.starsHighVAlpha  highVAlpha

                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }  
    }
    


    
}"""

SN2014JTemplate = """mesh
{  
    data center    ./modules/$folderName/center.raw
    data  quad ./modules/$folderName/quad.3ds
    
    cullRadius $cr
    glslVersion 330
    
    propertyCollection 
    {        
        __objectName__
        {           

            vec1f SNRadScale 5 | public | desc "max SN radius" | widget slider | range 0 100
            vec1f SNAlpha 1 | public | desc "SN alpha " | widget slider | range 0 1
            vec1f SNTeff 16000 | public | desc "Teff of for color" | widget slider | range 0 19000
            vec1f SNdlim 0. | public | desc "size of central solid bit" | widget slider | range 0 1
            vec1f SNdpow 0.9 | public | desc "exponential factor in falloff" | widget slider | range 0 10
            vec1f SNdfac 2.5 | public | desc "multiplicative factor in falloff" | widget slider | range 0 1
            
            vec1f eventTime 0 | public | desc "event time " #| widget slider | range 0 30
            vec1f transitionLength 5 | public | desc "transition length in seconds" 
            bool jump true | public  | desc "jump to time without transition"          
            
        }
    }
    
    ############# to hold the time information
    renderTexture
    {
        name                      stateTexture
        width                     1
        height                    1
        numTextures               1
        isPingPong                true
        isPersistent              true
        isFramePersistent         true
        internalTextureFormat     GL_RGB32F
        magnify                   GL_NEAREST
        minify                    GL_NEAREST
    }
    ############# set Transition State
    pass 
    {
        useDataObject quad
        renderTarget
        {
            name stateTexture
            enableColorClear false
        }
         shader
        {
            type defaultMeshShader
            {
                vertexShader   ./modules/$folderName/pass0.vs
                fragmentShader ./modules/$folderName/state.fs
                textureFBO stateTexture stateTexture
                stateManagerVar __objectName__.transitionLength transitionLength 
                stateManagerVar __objectName__.jump jump 
                stateManagerVar __objectName__.eventTime eventTime 
                parameter2f timeRange 0 100
           }
        }      
    }
    
    

    ############# SN (like stellar corona)   
    pass
    {
        useDataObject center
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/SNcorona.gs
                vertexShader ./modules/$folderName/SNcorona.vs
                fragmentShader   ./modules/$folderName/SNcorona.fs
                textureFBO stateTexture stateTexture
                texture bb ./modules/$folderName/bb.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    colorspace linear
                }
                
                stateManagerVar __objectName__.SNRadScale  radScale
                stateManagerVar __objectName__.SNTeff  Teff
                stateManagerVar __objectName__.SNAlpha  userAlpha
                stateManagerVar __objectName__.SNdlim  dlim
                stateManagerVar __objectName__.SNdpow  dpow
                stateManagerVar __objectName__.SNdfac  dfac
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }


}"""

### Galaxy and SN Classes

In [78]:
class GalaxyClass():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("FIREgas.raw",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("FIREstars.raw",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("galaxy.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("galaxy.fs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("galaxy.vs",dataFolder)
        self.cr = 1000
    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(GalaxyTemplate)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir, cr = self.cr))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)
        
class SNClass():
    def __init__(self, object):
        self.object = object        
        uvmod.Utility.ensurerelativepathexsists("SNcorona.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("SNcorona.fs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("SNcorona.vs",dataFolder)
        self.cr = 1000
    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(SN2014JTemplate)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir, cr = self.cr))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [79]:
Gmodel = GalaxyClass(uvmod.OrbitalObject())
Gscene = uvmod.Scene()
GparentScene = uvmod.Scene()
Gmodinfo = uvmod.ModuleInformation()
Ggenerator = uvmod.Generator()

In [80]:
Smodel = SNClass(uvmod.OrbitalObject())
Sscene = uvmod.Scene()
SparentScene = uvmod.Scene()
Smodinfo = uvmod.ModuleInformation()
Sgenerator = uvmod.Generator()

### Specify Settings and generate the module

In [81]:
Gscene.setname("M82FIRE")
Gscene.setparent("Extragalactic")
Gscene.setunit(5e18)
Gscene.setentrydist(1e5)
Gscene.setstaticposition(-2.07816693, 1.6584028, 2.27616396)

Gmodel.object.setcameraradius(1)
Gmodel.object.setcoord(Gscene.name)
Gmodel.object.setname("M82FIRE")
Gmodel.object.setguiname("/KavliLecture/Zooniverse/M82FIRE")
Gmodel.object.settargetradius(100)
Gmodel.object.showatstartup(False)
Gmodel.cr = 10

Gmodinfo.setname("M82FIRE")
Gmodinfo.setauthor("Aaron Geller<sup>1</sup>, Laura Trouille<sup>1</sup>, Chris Lintott<sup>2</sup> and the FIRE collaboration<sip>3</sup><br />(1)Adler Planetarium,<br />(2)Oxford University,<br />(3)https://fire.northwestern.edu/")
Gmodinfo.cleardependencies()
Gmodinfo.setdesc("Uniview module for a M82 using FIRE data")
Gmodinfo.setthumbnail("data/M82ScreenshotThumb.png")
Gmodinfo.setversion("1.0")

Ggenerator.generate("M82FIRE",[Gscene],[Gmodel],Gmodinfo)
uvmod.Utility.senduvcommand(Gmodel.object.name+".reload")


Unable to connect to Uniview


In [82]:
Sscene.setname("SN2014J")
Sscene.setparent("Extragalactic")
Sscene.setunit(5e18)
Sscene.setentrydist(1e5)
Sscene.setstaticposition(-2.07892522, 1.65798914, 2.27577283)

Smodel.object.setcameraradius(1)
Smodel.object.setcoord(Sscene.name)
Smodel.object.setname("SN2014J")
Smodel.object.setguiname("/KavliLecture/Zooniverse/SN2014J")
Smodel.object.settargetradius(100)
Smodel.object.showatstartup(False)
Smodel.cr = 10

Smodinfo.setname("SN2014J")
Smodinfo.setauthor("Aaron Geller<sup>1</sup>, Laura Trouille<sup>1</sup>, Chris Lintott<sup>2</sup> <br />(1)Adler Planetarium,<br />(2)Oxford University")
Smodinfo.cleardependencies()
Smodinfo.setdesc("Uniview module for a SN2014J to be used with the M82FIRE model")
#Smodinfo.setthumbnail("data/M82ScreenshotThumb.png")
Smodinfo.setversion("1.0")

Sgenerator.generate("SN2014J",[Sscene],[Smodel],Smodinfo)
uvmod.Utility.senduvcommand(Smodel.object.name+".reload")

Unable to connect to Uniview


## To reload

In [33]:
from config import Settings

In [34]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+Gmodel.object.name)
uvmod.Utility.senduvcommand(Gmodel.object.name+".reload;")
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+Smodel.object.name)
uvmod.Utility.senduvcommand(Smodel.object.name+".reload;")

### Create colormap texture

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

In [2]:
gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

def plot_cmap(colormap, fname):
    fig=plt.imshow(gradient, aspect=1, cmap=colormap)
    plt.axis('off')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.savefig(fname, bbox_inches='tight',pad_inches=0)


plot_cmap('Blues', 'data/cmapBlues.png')
plot_cmap('Reds', 'data/cmapReds.png')


## M82 position 

*From https://en.wikipedia.org/wiki/Messier_82*

In [1]:
from astropy.coordinates import SkyCoord, Distance
from astropy import units

In [2]:
RA = "9h 55m 52.2s" #right ascension
Dec= "69d 40m 47s" #declination
dist = (3500000 *units.pc).to(units.Mpc) #distance

coord = SkyCoord(RA, Dec, dist)

#print(coord.cartesian)
print(coord.galactic.cartesian)


(-2.07816693, 1.6584028, 2.27616396) Mpc


## Position  of SN 2014J in M82

https://en.wikipedia.org/wiki/SN_2014J

In [3]:
RA = "9h 55m 42.217s" #right ascension
Dec= "69d 40m 26.56s" #declination
dist = (3500000 *units.pc).to(units.Mpc) #distance

coord = SkyCoord(RA, Dec, dist)

print(coord.galactic.cartesian)

(-2.07892522, 1.65798914, 2.27577283) Mpc
